# Tuning Fufi hyperparameters with W&B Sweep 🧹 🐶
The idea is automatically tuning hyperparameters with the sweep by the means of a `grid `search.
In order not to spend an eternity doing this thing, we start with a` random` search, and then put a `grid `search on it.

## W&B Setup
🪄 Install `wandb` library and login

Start by installing the library and logging in to your free account.

In [1]:
!pip install wandb -qU
# Log in to your W&B account
import wandb
wandb.login()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 48.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.6/300.6 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.7 MB/s eta 0:00:00


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

## Installing libraries 📚

In [2]:
!pip install gym==0.25.2
#needed from March
!pip install numpy==1.23.5

## Setting things up for the environment 🌍 🪖

In [1]:
import os
#cloning Fufi repo from git
!git clone https://github.com/Gaianeve/gym-Fufi.git
#installing things
!pip install /content/gym-Fufi

fatal: destination path 'gym-Fufi' already exists and is not an empty directory.
Processing ./gym-Fufi
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [3]:
# Enter the environment directory
%cd /content/gym-Fufi
# Actually importing the library for our environment
import gym_Fufi

/content/gym-Fufi


In [4]:
#get back to content directory so I save everything there
%cd ..
!pwd

/content
/content


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## importing libraries and functions 📚


In [5]:
#libraries
import argparse
import random
import time
from distutils.util import strtobool
import gym
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torch.distributions.categorical import Categorical
from torch.utils.tensorboard import SummaryWriter
from torchsummary import summary

## Log needed function from files 📡
Loading files directly from git, so I don't have to upload it by hand.

In [6]:
#get files from git
!git clone https://github.com/Gaianeve/FUFONE.git

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Cloning into 'FUFONE'...
remote: Enumerating objects: 327, done.
remote: Counting objects: 100% (176/176), done.
remote: Compressing objects: 100% (76/76), done.
remote: Total 327 (delta 111), reused 162 (delta 99), pack-reused 151
Receiving objects: 100% (327/327), 9.76 MiB | 17.35 MiB/s, done.
Resolving deltas: 100% (185/185), done.


In [7]:
!pwd
%cd FUFONE/PPO
from environment import vectorize_env
from agent_class import Agent
from agent_utils import anneal, collect_data, GAE, PPO_train_agent, evaluate_agent

#back to content directory
%cd ..
%cd ..


/content
/content/FUFONE/PPO
/content/FUFONE
/content


In [8]:
#import main function from file
!pwd
%cd FUFONE/Sweep
from sweep_main import train_main, parse_args, get_default_hyperparameters_and_names

#back to content directory
%cd ..
%cd ..

/content
/content/FUFONE/Sweep
/content/FUFONE
/content


## Define the sweep 📑
I assume to start with a random search, that is why we take random distribution:
*   `ent_coef ` ➡ Discrete uniform distribution on integers.
*  ` num_envs`  ➡ list of values to try out
*   `learning_rate` ➡  see ent_coef
*  ` num_steps`  ➡ list of values to try out

In [9]:
import numpy as np
import random

# Define sweep config
sweep_configuration = {
    "method": "random",
    "name": "sweep_Fufi",
    "metric": {"goal": "maximize", "name": "sum_episodes"},
    "parameters": {
       "num_steps": {"values": [128, 256, 512, 2048, 4096, 8192]},
       "ent_coef": {'distribution': 'uniform', 'min': 0.01,'max': 1},
       "num_envs" : {"values": [2,4,8,16,32]},
       "learning_rate":  {'distribution': 'uniform', 'min': 1.5e-7,'max': 1.5e-4}
    },
}


In [10]:
#print the result
import pprint
pprint.pprint(sweep_configuration)

{'method': 'random',
 'metric': {'goal': 'maximize', 'name': 'sum_episodes'},
 'name': 'sweep_Fufi',
 'parameters': {'ent_coef': {'distribution': 'uniform', 'max': 1, 'min': 0.01},
                'learning_rate': {'distribution': 'uniform',
                                  'max': 0.00015,
                                  'min': 1.5e-07},
                'num_envs': {'values': [2, 4, 8, 16, 32]},
                'num_steps': {'values': [128, 256, 512, 2048, 4096, 8192]}}}


## Run main with the sweep 🏃 🧹
The `wandb.sweep` function initializes the sweep using the configuration. The `wandb.agent `function runs the sweep, executing the `sweep_main` function for each set of parameters.

📚 **Handling Parameters in Script**: In `sweep_main`, `wandb.init()` initializes a run. The script updates the args with the parameters from the sweep `(wandb.config)`, which are then passed to the main function.

📚 **Note**: Added `if __name__ == "__main__":` This ensures that main is called only when the script is executed directly, not when imported as a module.

In [11]:
import wandb
if __name__ == "__main__":
    sweep_id = wandb.sweep(sweep=sweep_configuration, project="Fufi_sweep")  # Set up the sweep

    #get default configuration
    default_config  = get_default_hyperparameters_and_names()

    def sweep_train():
      print(default_config)
      parse_args(default_config)
      train_main(default_config)


/usr/local/lib/python3.10/dist-packages/notebook/utils.py:280: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  return LooseVersion(v) >= LooseVersion(check)


Create sweep with ID: q7flojfh
Sweep URL: https://wandb.ai/cartpole_maria_gaia/Fufi_sweep/sweeps/q7flojfh


In [12]:
 wandb.agent(sweep_id, function=sweep_train, count = 50)

wandb: Agent Starting Run: 2dy9nk6e with config:
wandb: 	ent_coef: 0.3096027342320947
wandb: 	learning_rate: 1.6650237413872244e-05
wandb: 	num_envs: 16
wandb: 	num_steps: 256
ERROR:wandb.agents.pyagent:Run 2dy9nk6e errored:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/wandb/agents/pyagent.py", line 307, in _run_job
    self._function()
  File "<ipython-input-11-37bcd94df82f>", line 5, in sweep_train
    print(default_config)
NameError: name 'default_config' is not defined

wandb: ERROR Run 2dy9nk6e errored:
wandb: ERROR Traceback (most recent call last):
wandb: ERROR   File "/usr/local/lib/python3.10/dist-packages/wandb/agents/pyagent.py", line 307, in _run_job
wandb: ERROR     self._function()
wandb: ERROR   File "<ipython-input-11-37bcd94df82f>", line 5, in sweep_train
wandb: ERROR     print(default_config)
wandb: ERROR NameError: name 'default_config' is not defined
wandb: ERROR 
wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: 